In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
tf.logging.set_verbosity(tf.logging.INFO)

In [3]:
def cnn_model_fn(features, labels, mode):
    input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])
    
    conv1 = tf.layers.conv2d(
            inputs=input_layer,
            filters=32,
            kernel_size=[5,5],
            padding="same",
            activation=tf.nn.relu)
    
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], strides=2)
    
    conv2 = tf.layers.conv2d(
            inputs=pool1,
            filters=64,
            kernel_size=[5,5],
            padding="same",
            activation=tf.nn.relu)
    
    pool2 = tf.layers.max_pooling2d(conv2, pool_size=[2,2], strides=2)
    
    pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
    
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
    
    dropout = tf.layers.dropout(
            inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)
    
    logits = tf.layers.dense(inputs=dropout, units=10)
    
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)
    
    eval_metrics_op = {
        "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])
    }
    
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metrics_op=eval_metrics_op)

In [10]:
def main_():
    mnist = tf.contrib.learn.datasets.load_dataset("mnist")

    train_data = mnist.train.images
    train_labels = np.asarray(mnist.train.labels, dtype=np.int32)

    eval_data = mnist.test.images
    eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
    
    # Set up logging for predictions
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
        tensors=tensors_to_log, every_n_iter=50)
    
    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": train_data},
        y=train_labels,
        batch_size=100,
        num_epochs=None,
        shuffle=True)
    mnist_classifier.train(
        input_fn=train_input_fn,
        steps=200,
        hooks=[logging_hook])
    
    # Evaluate the model and print results
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": eval_data},
        y=eval_labels,
        num_epochs=1,
        shuffle=False)
    eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

In [6]:
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000225F1B839B0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [11]:
main_()

Extracting MNIST-data\train-images-idx3-ubyte.gz
Extracting MNIST-data\train-labels-idx1-ubyte.gz
Extracting MNIST-data\t10k-images-idx3-ubyte.gz
Extracting MNIST-data\t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into /tmp/mnist_convnet_model\model.ckpt.
INFO:tensorflow:probabilities = [[0.10139763 0.10511308 0.09826788 0.08887739 0.10208231 0.1100194
  0.09990142 0.09144492 0.10250697 0.10038893]
 [0.10349356 0.10252444 0.0950162  0.08802056 0.10834686 0.10240064
  0.08882475 0.10362773 0.10215785 0.10558735]
 [0.11032967 0.11170299 0.09925226 0.08354726 0.09478059 0.1114293
  0.0933888  0.08522838 0.11761002 0.09273069]
 [0.10884926 0.10514739 0.08450561 0.08141544 0.1122582  0.11231546
  0.0863736  0.08047058 0.12705167 0.101612

INFO:tensorflow:loss = 2.312137, step = 1
INFO:tensorflow:probabilities = [[0.1074141  0.1117212  0.09565675 0.09173673 0.10374971 0.10476131
  0.08647842 0.09018858 0.09949018 0.108803  ]
 [0.10278416 0.10957474 0.0922436  0.08269552 0.1040049  0.10556341
  0.0958125  0.09688772 0.11340256 0.09703093]
 [0.10760309 0.10672618 0.08893593 0.08657843 0.10856152 0.1001351
  0.09312271 0.09382473 0.1101606  0.10435173]
 [0.09803273 0.1086427  0.1094808  0.08814384 0.09771966 0.10931817
  0.08121072 0.09515107 0.11014153 0.10215881]
 [0.11154947 0.11170153 0.09702162 0.08350641 0.10485627 0.10336858
  0.08403555 0.08495533 0.12967096 0.08933435]
 [0.10744308 0.09902397 0.09652959 0.09459302 0.09995865 0.10185467
  0.09281363 0.1038259  0.10558049 0.09837704]
 [0.10137957 0.1150246  0.09026161 0.09210493 0.1013439  0.10689006
  0.08827879 0.09014481 0.10732973 0.10724205]
 [0.1083578  0.09582812 0.09836168 0.07945623 0.10591455 0.09808335
  0.09719991 0.09867334 0.11677433 0.10135067]
 [0.110

INFO:tensorflow:global_step/sec: 0.549413
INFO:tensorflow:probabilities = [[0.11171433 0.09845021 0.10171985 0.08544104 0.10678078 0.10329562
  0.09339622 0.09991539 0.10845135 0.09083523]
 [0.1028732  0.10660358 0.10116345 0.08834633 0.10175333 0.10052773
  0.09379607 0.09345097 0.11122135 0.10026402]
 [0.10863854 0.11424622 0.09475006 0.09403625 0.10219187 0.10712732
  0.09077953 0.09255426 0.10366033 0.0920156 ]
 [0.09494928 0.09966829 0.10148503 0.08892339 0.10215942 0.1011794
  0.09614166 0.10145741 0.1097057  0.10433047]
 [0.10876098 0.10094558 0.0938506  0.09317234 0.10614672 0.09396722
  0.09695577 0.09501448 0.11075592 0.10043037]
 [0.09967487 0.09270283 0.10321282 0.0917926  0.10291442 0.10233884
  0.0951336  0.09597582 0.11147578 0.10477833]
 [0.10469043 0.10575569 0.10396738 0.09293414 0.10730448 0.09853737
  0.09363104 0.08608719 0.11184517 0.09524707]
 [0.10705111 0.11296979 0.09958692 0.09101206 0.09725331 0.10586026
  0.09513651 0.08093277 0.10839745 0.10179988]
 [0.104

INFO:tensorflow:loss = 2.2953043, step = 101 (182.032 sec)
INFO:tensorflow:probabilities = [[0.11204443 0.09924637 0.10016949 0.09014455 0.10535666 0.1051195
  0.08831192 0.09861016 0.10578109 0.09521577]
 [0.10016499 0.10165463 0.10005286 0.09125596 0.10751105 0.09821768
  0.10168219 0.0976718  0.11152762 0.09026121]
 [0.11134533 0.07272674 0.10430416 0.08733243 0.10621961 0.10618699
  0.09761298 0.09387849 0.11835678 0.10203651]
 [0.09734323 0.09930652 0.10141947 0.09595827 0.09910058 0.10147978
  0.10160797 0.09133904 0.10948488 0.10296024]
 [0.09892197 0.09783936 0.09612407 0.0984682  0.10709308 0.10279957
  0.10327128 0.0995551  0.10689277 0.08903459]
 [0.10905711 0.09684181 0.10431299 0.09220838 0.100586   0.10645062
  0.09801237 0.09198108 0.11002947 0.09052011]
 [0.1017404  0.09510903 0.10540985 0.09389416 0.11114709 0.09749768
  0.09924769 0.09310103 0.1075366  0.09531645]
 [0.10322312 0.09582755 0.09867388 0.08216543 0.10260415 0.09985647
  0.09309238 0.08993576 0.12746727 0.

INFO:tensorflow:global_step/sec: 0.574257
INFO:tensorflow:probabilities = [[0.10203687 0.1019174  0.10705404 0.08757412 0.11039405 0.10077451
  0.08492327 0.093907   0.11742595 0.09399284]
 [0.11563531 0.09444819 0.09201384 0.10326545 0.09876756 0.10857515
  0.08824446 0.0958282  0.10366433 0.09955751]
 [0.11500612 0.09706199 0.10437084 0.09336884 0.1011837  0.10621598
  0.08653291 0.09647109 0.10298298 0.09680549]
 [0.1017781  0.10319892 0.09866706 0.09631363 0.10129837 0.1008569
  0.09686422 0.09539875 0.09716415 0.10845981]
 [0.10679961 0.09525035 0.10442731 0.0930711  0.10513687 0.10471722
  0.09789612 0.09422033 0.10427208 0.09420892]
 [0.11478177 0.08819579 0.090689   0.10405166 0.1066545  0.10237266
  0.09733951 0.08305435 0.11496205 0.09789865]
 [0.10337114 0.1078724  0.09127576 0.09024774 0.10286608 0.09281825
  0.09129445 0.09293799 0.1158092  0.11150704]
 [0.10334851 0.10957187 0.09709858 0.09078167 0.09987068 0.10325607
  0.1002812  0.08541733 0.11659084 0.09378324]
 [0.100

...

INFO:tensorflow:global_step/sec: 0.569539
INFO:tensorflow:probabilities = [[0.00014373 0.0188136  0.00951093 0.00205926 0.8138726  0.00394128
  0.10289119 0.00285761 0.02394536 0.02196443]
 [0.89802676 0.00000143 0.00069335 0.0110307  0.00001982 0.06614297
  0.00001262 0.01399769 0.00796495 0.00210967]
 [0.00012376 0.9819064  0.00092035 0.00187646 0.00015182 0.00061442
  0.00034505 0.0017461  0.01127443 0.00104121]
 [0.00057484 0.00128355 0.01522199 0.01097539 0.67892885 0.00152299
  0.0456172  0.00418524 0.08397229 0.15771781]
 [0.02752493 0.00008776 0.00053965 0.08076937 0.00121168 0.7552235
  0.00010861 0.00599869 0.04162253 0.08691334]
 [0.02113621 0.04990405 0.84931684 0.02228793 0.00006399 0.01507852
  0.03135838 0.00005111 0.01071402 0.00008896]
 [0.00142174 0.00032994 0.00283911 0.28879833 0.00068141 0.134168
  0.00007501 0.00073023 0.5628929  0.00806335]
 [0.00035767 0.96101975 0.00351277 0.00140966 0.0016381  0.00058554
  0.00149534 0.00032323 0.02959174 0.00006623]
 [0.01547

INFO:tensorflow:loss = 0.3467643, step = 5501 (175.584 sec)
INFO:tensorflow:probabilities = [[0.00150292 0.00000122 0.00055485 0.99618655 0.00000005 0.00064924
  0.00000325 0.00004465 0.00105468 0.00000269]
 [0.00008018 0.9489647  0.0015566  0.01511996 0.00118689 0.00098486
  0.00109282 0.01164309 0.00694986 0.01242096]
 [0.0001473  0.9711168  0.00072196 0.00263122 0.00037989 0.00206791
  0.00397281 0.00038493 0.01699479 0.00158224]
 [0.00020007 0.97183573 0.00196123 0.00085486 0.00014346 0.00019101
  0.00039994 0.00246826 0.02155499 0.00039037]
 [0.00968723 0.7796802  0.03157062 0.03627648 0.00479964 0.0099609
  0.01667685 0.01086553 0.08972384 0.01075866]
 [0.00000276 0.00000034 0.00000031 0.00000894 0.00296627 0.00000809
  0.00000058 0.00134337 0.00016395 0.99550533]
 [0.9980678  0.00000002 0.00029585 0.00002897 0.00000024 0.00100058
  0.00049002 0.00000365 0.00011113 0.00000168]
 [0.00032617 0.00000136 0.00000047 0.00101002 0.0001211  0.00116587
  0.00000012 0.9628421  0.00011476 0

INFO:tensorflow:global_step/sec: 0.576053
INFO:tensorflow:probabilities = [[0.00079841 0.00019324 0.00489013 0.00307587 0.00067359 0.00016663
  0.00002339 0.9313491  0.00205001 0.05677969]
 [0.00906705 0.00507827 0.00333024 0.0297123  0.79904    0.01425701
  0.00593435 0.00269843 0.03115045 0.09973192]
 [0.00012894 0.00000186 0.00000035 0.00082075 0.00110244 0.00101245
  0.00000035 0.2027802  0.00028942 0.7938632 ]
 [0.9860867  0.00000003 0.00019991 0.00006996 0.00000025 0.01328804
  0.00016977 0.00001151 0.00014787 0.00002598]
 [0.00003227 0.99311346 0.00063385 0.0019645  0.00030329 0.0005525
  0.00063162 0.00056172 0.00144441 0.00076214]
 [0.00050395 0.9130358  0.03072327 0.00561782 0.0058258  0.00156653
  0.00042309 0.00959225 0.03174946 0.0009621 ]
 [0.00000092 0.00015721 0.00205331 0.9953751  0.00000027 0.00180049
  0.00000028 0.00003746 0.00055667 0.00001841]
 [0.00970233 0.01360013 0.03495044 0.06729245 0.3356911  0.44403923
  0.02367099 0.01944283 0.01269056 0.03891994]
 [0.998

INFO:tensorflow:loss = 0.33293667, step = 5601 (173.546 sec)
INFO:tensorflow:probabilities = [[0.9804842  0.00000031 0.00002453 0.00067535 0.00044146 0.00307643
  0.0000338  0.00327996 0.00077545 0.01120851]
 [0.00018159 0.09671986 0.00101215 0.06718055 0.4969356  0.03448608
  0.0012932  0.10217415 0.03388231 0.16613464]
 [0.02072574 0.01177729 0.0007027  0.4109515  0.00369036 0.48451298
  0.02356553 0.00098764 0.03137157 0.01171455]
 [0.98461676 0.00000108 0.00023115 0.00068784 0.00000395 0.0100921
  0.00003771 0.00042681 0.00211799 0.00178469]
 [0.99976057 0.         0.00008303 0.00000059 0.         0.00014943
  0.00000184 0.00000001 0.00000434 0.00000023]
 [0.01005259 0.00166756 0.00057661 0.29424933 0.00014602 0.67261463
  0.00053868 0.0022652  0.01083995 0.00704943]
 [0.00297097 0.00002046 0.00005572 0.00005392 0.00011693 0.9097181
  0.08566496 0.00000008 0.00138668 0.00001214]
 [0.0000019  0.00000005 0.00003728 0.00060934 0.92079157 0.00162717
  0.00050914 0.02095157 0.00200817 0

KeyboardInterrupt: 